# MongoDB에서 바로 DataFrame으로 가져오기

In [1]:
# 방법 1: Binary UUID를 문자열로 변환하여 DataFrame에서 조회
from pymongo import MongoClient
import pandas as pd
import os
from dotenv import load_dotenv
from bson.binary import Binary
import uuid

load_dotenv(override=True)

# MongoDB 연결 정보
uri = os.getenv("MONGO_URI")
db_name = os.getenv("MONGO_DB_NAME")
collection_name = os.getenv("COLLECTION_NAME")

client = MongoClient(uri)
db = client[db_name]
collection = db[collection_name]

# MongoDB에서 데이터 불러오기
data = list(collection.find())

# DataFrame으로 변환
df = pd.DataFrame(data)

# 컬럼명 변경
df.rename(columns={"childId": "userId"}, inplace=True)
df.head()



,_id,investSessionId,chapterId,userId,turn,riskLevel,currentPoint,beforeValue,currentValue,initialValue,numberOfShares,income,transactionType,plusClick,minusClick,newsTag,startedAt,endedAt,_class
0,b'@F\xceR\xabbX\xf6tkI\x04\xfex\xa7\x89',b'}C\x99\xf5Z8\x86X\x07\x7f\xbe\xa4\xc8`=\xb3',1111,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,1,고위험 고수익,1400,0,100,100,0,0,KEEP,0,0,all,2025-06-19 09:58:33,2025-06-19 09:58:40,com.popoworld.backend.invest.entity.InvestHistory
1,b'\x11@\xad\x0b\xa0\xf2\xde$\xa0|\xf0\x99\xc7\...,b'}C\x99\xf5Z8\x86X\x07\x7f\xbe\xa4\xc8`=\xb3',1111,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,1,중위험 균형형,1400,0,100,100,0,0,KEEP,1,1,all,2025-06-19 09:58:33,2025-06-19 09:58:40,com.popoworld.backend.invest.entity.InvestHistory
2,b'\xf5H \xcc\xbd\x8f6\xd6\x02\x85u\xf8\xcf\x12...,b'}C\x99\xf5Z8\x86X\x07\x7f\xbe\xa4\xc8`=\xb3',1111,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,1,저위험 저수익,1400,0,100,100,6,0,BUY,6,0,all,2025-06-19 09:58:33,2025-06-19 09:58:40,com.popoworld.backend.invest.entity.InvestHistory
3,b'\xf9I\xf7oN\x1c\xc7\xaf\xde\xd0\t@\x03\xd4\x...,b'}C\x99\xf5Z8\x86X\x07\x7f\xbe\xa4\xc8`=\xb3',1111,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,2,중위험 균형형,848,100,96,100,0,0,KEEP,0,0,high,2025-06-19 09:58:40,2025-06-19 09:58:45,com.popoworld.backend.invest.entity.InvestHistory
4,b'DO\x1d\x7f<\x0cw\x9f`!\xcb\xca\x9cW1\xb7',b'}C\x99\xf5Z8\x86X\x07\x7f\xbe\xa4\xc8`=\xb3',1111,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,2,고위험 고수익,848,100,92,100,6,0,BUY,6,0,high,2025-06-19 09:58:40,2025-06-19 09:58:45,com.popoworld.backend.invest.entity.InvestHistory


In [3]:
df["chapterId"].value_counts()

chapterId
1111    36
4444    18
Name: count, dtype: int64

In [3]:
def mongo_preprocess(df):

    df['riskLevel'] = df['riskLevel'].replace({
        '고위험 고수익': 'high',
        '중위험 균형형': 'mid',
        '저위험 저수익': 'low'
    })

    df.sort_values(by=['investSessionId', 'riskLevel', 'turn'], inplace=True)
    df['deltaShares'] = df.groupby(['investSessionId', 'riskLevel'])['numberOfShares'].diff()
    df['deltaShares'] = df['deltaShares'].fillna(df['numberOfShares'])
    df['deltaShares'] = df['deltaShares'].astype(int)

    return df

df = mongo_preprocess(df)
df.head()

,_id,investSessionId,chapterId,userId,turn,riskLevel,currentPoint,beforeValue,currentValue,initialValue,numberOfShares,income,transactionType,plusClick,minusClick,newsTag,startedAt,endedAt,_class,deltaShares
132,"b'\xc5KAK""M96\xe5\xcbR4\xa1\x8b\x19\x9f'",b'\x1cM\xa2\x9b\xc6\xa8\x90A\xd1\xbd\x13\xc8aE...,1111,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,1,high,2000,0,100,100,0,0,KEEP,0,0,all,2025-06-18 13:08:47,2025-06-18 13:08:50,com.popoworld.backend.invest.entity.InvestHistory,0
144,b'MFwdh\xec\xc0\xcb\xbe^\xdfT\xads\x94\x8c',b'\x1cM\xa2\x9b\xc6\xa8\x90A\xd1\xbd\x13\xc8aE...,1111,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,2,high,2000,100,92,100,0,0,KEEP,0,0,high,2025-06-18 13:08:50,2025-06-18 13:09:54,com.popoworld.backend.invest.entity.InvestHistory,0
149,b'\xf6Dl%\x14\xf0Zl\x94\x89\x06I\xd3\x1c\xc1\xa8',b'\x1cM\xa2\x9b\xc6\xa8\x90A\xd1\xbd\x13\xc8aE...,1111,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,3,high,2000,92,77,100,0,0,KEEP,0,0,mid,2025-06-18 13:09:54,2025-06-18 13:09:56,com.popoworld.backend.invest.entity.InvestHistory,0
150,b'rJB\xe0i\xfbP\x7f\xde\xbfy\x8f\x81\x93\xd9\x9f',b'\x1cM\xa2\x9b\xc6\xa8\x90A\xd1\xbd\x13\xc8aE...,1111,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,4,high,2000,77,78,100,0,0,KEEP,0,0,all,2025-06-18 13:09:56,2025-06-18 13:09:59,com.popoworld.backend.invest.entity.InvestHistory,0
153,b'\xb8JG\xc6\x9d\xc76^\x12\xa1Sx\xfa\xd2\xc8\xb5',b'\x1cM\xa2\x9b\xc6\xa8\x90A\xd1\xbd\x13\xc8aE...,1111,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,5,high,2000,78,87,100,0,0,KEEP,0,0,high,2025-06-18 13:09:59,2025-06-18 13:10:20,com.popoworld.backend.invest.entity.InvestHistory,0


# PostgreSQL에서 DataFrame으로 가져오기

In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os
import psycopg2

load_dotenv(override=True)

# PostgreSQL 연결 정보
conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),          
    dbname=os.getenv("DB_NAME"), 
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD") 
)

# SQL 쿼리 실행 후 DataFrame으로 불러오기
s_query = "SELECT chapter_id, seed_money FROM invest_chapter;"
c_query = "SELECT user_id, age, sex FROM users;"
seed = pd.read_sql(s_query, conn)
child = pd.read_sql(c_query, conn)

# 연결 종료
conn.close()

# 결과 확인
print(seed.head())
print(child.head())

  chapter_id  seed_money
0       1111         700
1       2222        2000
2       3333        5000
3       4444       10000
                                user_id  age                 created_at  \
0  969e5004-542c-47c1-b906-d81b907c5d08   39 2025-06-13 06:26:51.586245   
1  d97a07eb-8dd9-4cbe-a4c4-8c61017ada71    6 2025-06-13 06:27:46.843005   
2  c1194c9c-1ca4-435b-950e-ad68be4ba92f   22 2025-06-13 08:00:05.104626   
3  74f9a8f9-4f95-4759-ba5f-106040e2993a   24 2025-06-13 08:00:56.477722   
4  6544af34-fc16-412c-aa7a-b3eda54ebe61   23 2025-06-13 08:01:25.902605   

                   email          name parent_code  \
0  testparent1@naver.com    popoparent    2c43d81e   
1   testchild1@naver.com     popochild    2c43d81e   
2           tt1@test.com  tt1@test.com    6e4248a5   
3           tt2@test.com  tt2@test.com    b6c530e3   
4           tt3@test.com  tt3@test.com    01ac33d1   

                                            password  point    role sex  \
0  $2a$10$gmSf0f4l4z3lH7

C:\Users\user\AppData\Local\Temp\ipykernel_17244\4180636186.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  seed = pd.read_sql(s_query, conn)
C:\Users\user\AppData\Local\Temp\ipykernel_17244\4180636186.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  child = pd.read_sql(c_query, conn)


In [9]:
# PostgreSQL과 MongoDB의 변수명 통일
seed.rename(columns={'chapter_id': 'chapterId'}, inplace=True)
seed.rename(columns={'seed_money': 'seedMoney'}, inplace=True)
print(seed.head())

# PostgreSQL과 MongoDB의 변수명 통일
child.rename(columns={'user_id':'userId'}, inplace=True)
child.head()

  chapterId  seedMoney
0      1111        700
1      2222       2000
2      3333       5000
3      4444      10000


,userId,age,created_at,email,name,parent_code,password,point,role,sex,parent_id
0,969e5004-542c-47c1-b906-d81b907c5d08,39,2025-06-13 06:26:51.586245,testparent1@naver.com,popoparent,2c43d81e,$2a$10$gmSf0f4l4z3lH7uU7culGOgPrdQLzQNHqGUK42s...,0,Parent,F,None
1,d97a07eb-8dd9-4cbe-a4c4-8c61017ada71,6,2025-06-13 06:27:46.843005,testchild1@naver.com,popochild,2c43d81e,$2a$10$OfnFV0jAX83WuV2w7FdrPeqMUvEBOU.dDqwBfjd...,10000,Child,F,969e5004-542c-47c1-b906-d81b907c5d08
2,c1194c9c-1ca4-435b-950e-ad68be4ba92f,22,2025-06-13 08:00:05.104626,tt1@test.com,tt1@test.com,6e4248a5,$2a$10$iN8jr2EFnF54gBsiv9yXguvhamTIWY82WfwcEHL...,0,Parent,M,None
3,74f9a8f9-4f95-4759-ba5f-106040e2993a,24,2025-06-13 08:00:56.477722,tt2@test.com,tt2@test.com,b6c530e3,$2a$10$oJBOy3SSfHW5zeiL7oP5buCCXCFvmD2K8jzX04F...,0,Parent,M,None
4,6544af34-fc16-412c-aa7a-b3eda54ebe61,23,2025-06-13 08:01:25.902605,tt3@test.com,tt3@test.com,01ac33d1,$2a$10$gAZMt0PMqXLcx69.ejMXquyNDTkE0EVHe4ecyrD...,0,Parent,M,None


# mongoDB + SQL

In [6]:
df = df.merge(child, on="userId", how="left")

# 거래참여 비율 (tradingTurn / investSessionId + childId)

In [7]:
# 거래참여 턴 비율
def trading_turn(df):
    tpt = df[['investSessionId', 
          'userId',
          'turn',
          'riskLevel',
          'transactionType']]
    
    hold_flags = tpt.groupby(['investSessionId', 'userId', 'turn'])['transactionType'].apply(lambda x: int((x != "KEEP").any())).reset_index(name="notAllKeep")
    tradingTurn = hold_flags.groupby(['investSessionId', 'userId'])['notAllKeep'].mean().reset_index(name="avgNotKeep")
    
    return tradingTurn


tradingTurn = trading_turn(df)
tradingTurn.head()

,investSessionId,userId,avgNotKeep
0,b'\x1cM\xa2\x9b\xc6\xa8\x90A\xd1\xbd\x13\xc8aE...,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,0.0
1,b' J\xe4QK<\x7f\xdb\x12K\xc2\x1c\xd3.\xa6\xa1',237aac1b-4d6f-4ca9-9e4f-30719ea5967d,1.0
2,"b'""O\xcf1){(\x13\x9c_k[\xf4i\xcc\xa0'",f0220d43-513a-4619-973d-4ed84a42bf6a,1.0
3,b'>N\xea\x92\x1eT\x04\xcb\xeeC\xf6V\xa6\xbd\x9...,956f51a8-d6a0-4a12-a22b-9da3cdffc879,1.0
4,b'mB\xda\xce\xd4\x97\xfc\x83\xbd)\xa9 1t\x98\xab',237aac1b-4d6f-4ca9-9e4f-30719ea5967d,1.0


# 거래횟수 (transactionNum / investSessionId)

In [8]:
# 거래 횟수
def transaction_num(df):
    nt = df[['investSessionId',
             'userId' ,
          'turn',
          'riskLevel',
          'plusClick',
          'minusClick']].copy()

    nt['click'] = nt['plusClick'] + nt['minusClick']
    transactionNum = nt.groupby(['investSessionId','userId'])[['click']].mean().reset_index()

    return transactionNum

transactionNum = transaction_num(df)

In [9]:
transactionNum.head()

,investSessionId,userId,click
0,b'\x1cM\xa2\x9b\xc6\xa8\x90A\xd1\xbd\x13\xc8aE...,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,0.000000
1,b' J\xe4QK<\x7f\xdb\x12K\xc2\x1c\xd3.\xa6\xa1',237aac1b-4d6f-4ca9-9e4f-30719ea5967d,1.066667
2,"b'""O\xcf1){(\x13\x9c_k[\xf4i\xcc\xa0'",f0220d43-513a-4619-973d-4ed84a42bf6a,0.944444
3,b'>N\xea\x92\x1eT\x04\xcb\xeeC\xf6V\xa6\xbd\x9...,956f51a8-d6a0-4a12-a22b-9da3cdffc879,1.222222
4,b'mB\xda\xce\xd4\x97\xfc\x83\xbd)\xa9 1t\x98\xab',237aac1b-4d6f-4ca9-9e4f-30719ea5967d,1.333333


# 평균 현금잔여율 (averageCashRatio / investSessionId)

In [10]:
merged = pd.merge(df, seed, on="chapterId", how="left")

merged.head()

,_id,investSessionId,chapterId,userId,turn,riskLevel,currentPoint,beforeValue,currentValue,initialValue,...,plusClick,minusClick,newsTag,startedAt,endedAt,_class,deltaShares,age,sex,seedMoney
0,"b'\xc5KAK""M96\xe5\xcbR4\xa1\x8b\x19\x9f'",b'\x1cM\xa2\x9b\xc6\xa8\x90A\xd1\xbd\x13\xc8aE...,1111,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,1,high,2000,0,100,100,...,0,0,all,2025-06-18 13:08:47,2025-06-18 13:08:50,com.popoworld.backend.invest.entity.InvestHistory,0,24,M,700
1,b'MFwdh\xec\xc0\xcb\xbe^\xdfT\xads\x94\x8c',b'\x1cM\xa2\x9b\xc6\xa8\x90A\xd1\xbd\x13\xc8aE...,1111,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,2,high,2000,100,92,100,...,0,0,high,2025-06-18 13:08:50,2025-06-18 13:09:54,com.popoworld.backend.invest.entity.InvestHistory,0,24,M,700
2,b'\xf6Dl%\x14\xf0Zl\x94\x89\x06I\xd3\x1c\xc1\xa8',b'\x1cM\xa2\x9b\xc6\xa8\x90A\xd1\xbd\x13\xc8aE...,1111,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,3,high,2000,92,77,100,...,0,0,mid,2025-06-18 13:09:54,2025-06-18 13:09:56,com.popoworld.backend.invest.entity.InvestHistory,0,24,M,700
3,b'rJB\xe0i\xfbP\x7f\xde\xbfy\x8f\x81\x93\xd9\x9f',b'\x1cM\xa2\x9b\xc6\xa8\x90A\xd1\xbd\x13\xc8aE...,1111,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,4,high,2000,77,78,100,...,0,0,all,2025-06-18 13:09:56,2025-06-18 13:09:59,com.popoworld.backend.invest.entity.InvestHistory,0,24,M,700
4,b'\xb8JG\xc6\x9d\xc76^\x12\xa1Sx\xfa\xd2\xc8\xb5',b'\x1cM\xa2\x9b\xc6\xa8\x90A\xd1\xbd\x13\xc8aE...,1111,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,5,high,2000,78,87,100,...,0,0,high,2025-06-18 13:09:59,2025-06-18 13:10:20,com.popoworld.backend.invest.entity.InvestHistory,0,24,M,700


In [11]:
def avg_cash_ratio(df):
    acr = df[['investSessionId', 
              'userId',
              'seedMoney',
            'chapterId',
            'turn',
            'currentPoint']].copy()

    acr = acr.groupby(['investSessionId', 'userId', 'seedMoney', 'chapterId'])['currentPoint'].mean().reset_index(name='avgCurrentValue')
    acr['diff'] = acr['seedMoney'] - acr['avgCurrentValue']
    acr["avgCashRatio"] = acr["diff"] / acr["seedMoney"]

    acr = acr[["investSessionId","userId","avgCashRatio"]]
        
    return acr

avgCashRatio = avg_cash_ratio(merged)
avgCashRatio.head()

,investSessionId,userId,avgCashRatio
0,b'\x1cM\xa2\x9b\xc6\xa8\x90A\xd1\xbd\x13\xc8aE...,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,-1.857143
1,b' J\xe4QK<\x7f\xdb\x12K\xc2\x1c\xd3.\xa6\xa1',237aac1b-4d6f-4ca9-9e4f-30719ea5967d,-0.753714
2,"b'""O\xcf1){(\x13\x9c_k[\xf4i\xcc\xa0'",f0220d43-513a-4619-973d-4ed84a42bf6a,-0.235952
3,b'>N\xea\x92\x1eT\x04\xcb\xeeC\xf6V\xa6\xbd\x9...,956f51a8-d6a0-4a12-a22b-9da3cdffc879,-0.548333
4,b'mB\xda\xce\xd4\x97\xfc\x83\xbd)\xa9 1t\x98\xab',237aac1b-4d6f-4ca9-9e4f-30719ea5967d,0.680000


# 평균 턴 체류시간 (avgStayTime / investSessionId)

In [ ]:
def avg_stay_time(df):
    turn = df[['investSessionId', 'userId', 'age', 'turn', 'startedAt', 'endedAt']].copy()

    # 문자열을 datetime으로 변환
    turn['startedAt'] = pd.to_datetime(turn['startedAt'])
    turn['endedAt'] = pd.to_datetime(turn['endedAt'])

    # 체류 시간 계산
    turn['stayTime'] = (turn['endedAt'] - turn['startedAt']).dt.total_seconds()

    # 평균 체류 시간 계산
    avg_time = (
        turn
        .drop_duplicates(subset=['investSessionId', 'turn'])
        .groupby(['investSessionId', 'userId', 'age'])['stayTime']
        .mean()
        .reset_index(name='avgStayTime')
    )

    # 각 그룹의 첫 startedAt 추출
    first_start = (
        turn
        .sort_values(by='turn')
        .groupby(['investSessionId', 'userId', 'age'])
        .first()
        .reset_index()[['investSessionId', 'userId', 'age', 'startedAt']]
    )

    # 평균 체류 시간과 첫 startedAt 병합
    result = pd.merge(avg_time, first_start, on=['investSessionId', 'userId', 'age'])

    return result

avgStayTime = avg_stay_time(df)
avgStayTime.head()

,investSessionId,userId,age,avgStayTime,startedAt
0,b'\x1cM\xa2\x9b\xc6\xa8\x90A\xd1\xbd\x13\xc8aE...,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,24,17.500000,2025-06-18 13:08:47
1,b' J\xe4QK<\x7f\xdb\x12K\xc2\x1c\xd3.\xa6\xa1',237aac1b-4d6f-4ca9-9e4f-30719ea5967d,24,3.800000,2025-06-18 14:12:26
2,"b'""O\xcf1){(\x13\x9c_k[\xf4i\xcc\xa0'",f0220d43-513a-4619-973d-4ed84a42bf6a,24,48.833333,2025-06-18 13:01:58
3,b'>N\xea\x92\x1eT\x04\xcb\xeeC\xf6V\xa6\xbd\x9...,956f51a8-d6a0-4a12-a22b-9da3cdffc879,24,22.833333,2025-06-18 14:54:34
4,b'mB\xda\xce\xd4\x97\xfc\x83\xbd)\xa9 1t\x98\xab',237aac1b-4d6f-4ca9-9e4f-30719ea5967d,24,7.000000,2025-06-18 13:06:20
5,b'oD?\xe6a$w\xc7Br\x02~c)\x89\xa3',f0220d43-513a-4619-973d-4ed84a42bf6a,24,6.000000,2025-06-18 12:58:14
6,b'\x83H}\xe9;\x86$<le\x02\xb9C\x116\xb0',f0220d43-513a-4619-973d-4ed84a42bf6a,24,13.166667,2025-06-18 12:56:44
7,"b""\x84L\xeb\xb7*>7\xdcu'+|\x95\xc6&\x93""",956f51a8-d6a0-4a12-a22b-9da3cdffc879,24,14.500000,2025-06-18 14:51:52
8,b'\x8fCj\x9a\xb0\x04[~2d\xb2\x96\xf9x\x12\xba',956f51a8-d6a0-4a12-a22b-9da3cdffc879,24,7.166667,2025-06-18 14:44:09
9,b'\x94@\x84\xdb\x006\x9c\xa5\xed\x84N\xadZs\xb...,f0220d43-513a-4619-973d-4ed84a42bf6a,24,18.166667,2025-06-18 12:59:56


# 각 종목 별 구매/판매 비율 (avgTradeRatio / InvestSessionId)

In [13]:
import numpy as np
# 각 위험 별 구매/판매 비율
def avg_trade_ratio(df):
    # 구매 데이터 생성
    transcation_df = df[['investSessionId',
                        'userId',
                        'turn',
                        'riskLevel',
                        'numberOfShares',
                        'deltaShares']]
    
    # 변화량에 따라 BUY/SELL 구분하기
    transcation_df['is_buy'] = np.where(
        transcation_df['deltaShares'] > 0, 
        transcation_df['deltaShares'], 
        0
    )

    transcation_df['is_sell'] = np.where(
        transcation_df['deltaShares'] < 0, 
        -transcation_df['deltaShares'],
        0
    )

    # 각 investSessionId, riskLevel 별로 buy/sell 개수 집계
    buy_sell_counts = transcation_df.groupby(['investSessionId', 'riskLevel'])[['is_buy', 'is_sell']].sum().reset_index()
    buy_sell_counts.rename(columns={'is_buy': 'buyCount', 'is_sell': 'sellCount'}, inplace=True)


    # investSessionId별 전체 buy_count, sell_count 구하기
    total_buy = buy_sell_counts.groupby('investSessionId')['buyCount'].sum().reset_index(name='totalBuyCount')
    total_sell = buy_sell_counts.groupby('investSessionId')['sellCount'].sum().reset_index(name='totalSellCount')

    # 원본과 병합
    buy_sell_counts = buy_sell_counts.merge(total_buy, on='investSessionId')
    buy_sell_counts = buy_sell_counts.merge(total_sell, on='investSessionId')

    buy_sell_counts['buyRatio'] = buy_sell_counts['buyCount'] / buy_sell_counts['totalBuyCount']
    buy_sell_counts['sellRatio'] = buy_sell_counts['sellCount'] / buy_sell_counts['totalSellCount']

    buy_sell_counts.head()

    # buy_ratio_pivot: 열로 riskLevel을 펼치기  
    buy_ratio_pivot = buy_sell_counts.pivot(index='investSessionId', columns='riskLevel', values='buyRatio')
    buy_ratio_pivot = buy_ratio_pivot.fillna(0)
    buy_ratio_pivot.columns = [f"{level}BuyRatio" for level in buy_ratio_pivot.columns]
    buy_ratio_pivot.reset_index(inplace=True)

    # sell_ratio pivot: 열로 riskLevel을 펼치기
    sell_ratio_pivot = buy_sell_counts.pivot(index='investSessionId', columns='riskLevel', values='sellRatio')
    sell_ratio_pivot = sell_ratio_pivot.fillna(0)
    sell_ratio_pivot.columns = [f"{level}SellRatio" for level in sell_ratio_pivot.columns]
    sell_ratio_pivot.reset_index(inplace=True)

    # merge 두 pivot
    avgTradeRatio = pd.merge(buy_ratio_pivot, sell_ratio_pivot, on='investSessionId')

    user_info = transcation_df.groupby('investSessionId')[['userId']].first().reset_index()
    avgTradeRatio = avgTradeRatio.merge(user_info, on='investSessionId', how='left')

    return avgTradeRatio

avgTradeRatio = avg_trade_ratio(df)
avgTradeRatio.head()

C:\Users\user\AppData\Local\Temp\ipykernel_8312\719790905.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcation_df['is_buy'] = np.where(
C:\Users\user\AppData\Local\Temp\ipykernel_8312\719790905.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcation_df['is_sell'] = np.where(


,investSessionId,highBuyRatio,lowBuyRatio,midBuyRatio,highSellRatio,lowSellRatio,midSellRatio,userId
0,b'\x1cM\xa2\x9b\xc6\xa8\x90A\xd1\xbd\x13\xc8aE...,0.000000,0.000000,0.000000,0.00,0.0,0.00,237aac1b-4d6f-4ca9-9e4f-30719ea5967d
1,b' J\xe4QK<\x7f\xdb\x12K\xc2\x1c\xd3.\xa6\xa1',0.500000,0.388889,0.111111,1.00,0.0,0.00,237aac1b-4d6f-4ca9-9e4f-30719ea5967d
2,"b'""O\xcf1){(\x13\x9c_k[\xf4i\xcc\xa0'",0.588235,0.176471,0.235294,0.00,0.0,0.00,f0220d43-513a-4619-973d-4ed84a42bf6a
3,b'>N\xea\x92\x1eT\x04\xcb\xeeC\xf6V\xa6\xbd\x9...,0.166667,0.555556,0.277778,0.25,0.0,0.75,956f51a8-d6a0-4a12-a22b-9da3cdffc879
4,b'mB\xda\xce\xd4\x97\xfc\x83\xbd)\xa9 1t\x98\xab',0.000000,1.000000,0.000000,0.00,0.0,0.00,237aac1b-4d6f-4ca9-9e4f-30719ea5967d


# 평균 tag 뉴스 발생 턴 체류 시간 (tagAvgStayTime / investSessionId)

In [14]:
# 평균 tag 뉴스 발생 턴 체류 시간
def tag_avg_stay_time(df):
    tag_turn_df = df[['investSessionId',
                        'userId',
                        'turn',
                        'newsTag',
                        'startedAt',
                        'endedAt']].drop_duplicates()

    tag_turn_df = tag_turn_df[tag_turn_df['newsTag'] != "all"]
    tag_turn_df["turnDuration"] = tag_turn_df["endedAt"] - tag_turn_df["startedAt"]
    tagAvgStayTime = tag_turn_df.groupby(["investSessionId", 'userId'])["turnDuration"].mean().reset_index().rename(columns={"turnDuration":"tagTrunDuraion"})

    return tagAvgStayTime

tagAvgStayTime = tag_avg_stay_time(df)
tagAvgStayTime.head()

,investSessionId,userId,tagTrunDuraion
0,b'\x1cM\xa2\x9b\xc6\xa8\x90A\xd1\xbd\x13\xc8aE...,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,0 days 00:00:24.750000
1,b' J\xe4QK<\x7f\xdb\x12K\xc2\x1c\xd3.\xa6\xa1',237aac1b-4d6f-4ca9-9e4f-30719ea5967d,0 days 00:00:04
2,"b'""O\xcf1){(\x13\x9c_k[\xf4i\xcc\xa0'",f0220d43-513a-4619-973d-4ed84a42bf6a,0 days 00:01:07.250000
3,b'>N\xea\x92\x1eT\x04\xcb\xeeC\xf6V\xa6\xbd\x9...,956f51a8-d6a0-4a12-a22b-9da3cdffc879,0 days 00:00:10.250000
4,b'mB\xda\xce\xd4\x97\xfc\x83\xbd)\xa9 1t\x98\xab',237aac1b-4d6f-4ca9-9e4f-30719ea5967d,0 days 00:00:07


# 위험 감수율 (betMidShares, betHighShares / investSessionId)
- 중위험 tag 발생 시 중위험 종목 구매 수량
- 고위험 tag 발생 시 고위험 종목 구매 수량

In [15]:
def bet_shares(df):
    bet_data = df[["investSessionId","turn","userId","newsTag","riskLevel","numberOfShares","deltaShares", "transactionType", "beforeValue","currentValue","income"]].copy()

    # newsTag 발생 시 해당 종목을 구매한 경우
    bet_buy = bet_data.loc[(bet_data["newsTag"] == bet_data["riskLevel"]) & (bet_data["transactionType"]=="BUY")]

    bet_mid = bet_buy[bet_buy["riskLevel"]=="mid"].groupby(["investSessionId","userId"])["deltaShares"].sum().reset_index().rename(columns={"deltaShares":"betMidShares"})
    bet_high = bet_buy[bet_buy["riskLevel"]=="high"].groupby(["investSessionId","userId"])["deltaShares"].sum().reset_index().rename(columns={"deltaShares":"betHighShares"})

    risk_taking = pd.merge(bet_mid, bet_high, on=["investSessionId","userId"], how="outer")

    # nan 인 값들은 0으로 채우기
    risk_taking = risk_taking.fillna(0)

    return risk_taking

betShares = bet_shares(df)
betShares.head()

,investSessionId,userId,betMidShares,betHighShares
0,"b'""O\xcf1){(\x13\x9c_k[\xf4i\xcc\xa0'",f0220d43-513a-4619-973d-4ed84a42bf6a,4,4
1,b'>N\xea\x92\x1eT\x04\xcb\xeeC\xf6V\xa6\xbd\x9...,956f51a8-d6a0-4a12-a22b-9da3cdffc879,4,2
2,b'oD?\xe6a$w\xc7Br\x02~c)\x89\xa3',f0220d43-513a-4619-973d-4ed84a42bf6a,7,5
3,b'\x83H}\xe9;\x86$<le\x02\xb9C\x116\xb0',f0220d43-513a-4619-973d-4ed84a42bf6a,6,7
4,"b""\x84L\xeb\xb7*>7\xdcu'+|\x95\xc6&\x93""",956f51a8-d6a0-4a12-a22b-9da3cdffc879,3,3


# 베팅 성공률 (betBuyRatio, betSellRatio / investSessionId)
: 베팅 성공률
1) 구매 베팅 성공 = tag뉴스 발생 턴에서 해당 종목 구매 후 다음 턴에서 가격이 증가한 횟수 / tag 뉴스 발생턴에서 해당 종목을 구매한 횟수
2) 판매 베팅 성공 = tag뉴스 발생 턴에서 해당 종목 판매 후 다음 턴에서 가격이 감소한 횟수 / tag 뉴스 발생턴에서 해당 종목을 판매한 횟수


In [16]:
def bet_buy_ratio(df):
    # 구매 베팅 성공
    bet_win = df[["investSessionId",
                  "userId",
                  "turn",
                  "newsTag",
                  "riskLevel",
                  "beforeValue",
                  "currentValue", 
                  "transactionType"]].copy()

    # 다음 턴의 value 컬럼 구하기
    bet_win.sort_values(by=["investSessionId","riskLevel","turn"], inplace=True)
    bet_win["nextValue"] = bet_win["currentValue"].shift(-1)

    # tag 뉴스 턴에서 해당 종목을 구매한 횟수
    bet_buy = bet_win.loc[(bet_win["newsTag"]==bet_win["riskLevel"]) & (bet_win["transactionType"]=="BUY")].copy()
    bet_buy_total = bet_buy.groupby("investSessionId")["nextValue"].count().reset_index().rename(columns={"nextValue":"bet_buy_total"})

    # tag 뉴스 턴에서 해당 종목을 구매하고 다음 턴에서 가격이 오른 횟수
    bet_buy["value_diff"] = bet_buy["nextValue"] - bet_buy["currentValue"]
    bet_buy_win = bet_buy[bet_buy["value_diff"]>0]
    bet_buy_win = bet_buy_win.groupby("investSessionId")["value_diff"].count().reset_index().rename(columns={"value_diff":"bet_buy_win"})

    # 성공 비율 계산
    bet_buy_df = pd.merge(bet_buy_total, bet_buy_win, on="investSessionId", how="left")
    bet_buy_df["betBuyRatio"] = bet_buy_df["bet_buy_win"] / bet_buy_df["bet_buy_total"]

    # nan값 0으로 채우기
    bet_buy_df = bet_buy_df.fillna(0) # 데이터가 없어서 nan으로 출력됨..! 0으로 채워주기기

    # 필요없는 컬럼 삭제
    bet_buy_df.drop(columns=["bet_buy_total","bet_buy_win"], inplace=True)

    user_info = df.groupby("investSessionId")[["userId"]].first().reset_index()
    bet_buy_df = pd.merge(bet_buy_df, user_info, on="investSessionId", how="left")
    
    return bet_buy_df

betBuyRatio = bet_buy_ratio(df)
betBuyRatio.head()

,investSessionId,betBuyRatio,userId
0,"b'""O\xcf1){(\x13\x9c_k[\xf4i\xcc\xa0'",0.333333,f0220d43-513a-4619-973d-4ed84a42bf6a
1,b'>N\xea\x92\x1eT\x04\xcb\xeeC\xf6V\xa6\xbd\x9...,0.666667,956f51a8-d6a0-4a12-a22b-9da3cdffc879
2,b'oD?\xe6a$w\xc7Br\x02~c)\x89\xa3',0.333333,f0220d43-513a-4619-973d-4ed84a42bf6a
3,b'\x83H}\xe9;\x86$<le\x02\xb9C\x116\xb0',0.500000,f0220d43-513a-4619-973d-4ed84a42bf6a
4,"b""\x84L\xeb\xb7*>7\xdcu'+|\x95\xc6&\x93""",0.666667,956f51a8-d6a0-4a12-a22b-9da3cdffc879


In [17]:
def bet_sell_ratio(df):
    # 판매 베팅 성공
    bet_win = df[["investSessionId",
                  "userId",
                  "turn",
                  "newsTag",
                  "riskLevel",
                  "beforeValue",
                  "currentValue", 
                  "transactionType"]].copy()

    # 다음 턴의 value 컬럼 구하기
    bet_win.sort_values(by=["investSessionId","riskLevel","turn"], inplace=True)
    bet_win["nextValue"] = bet_win["currentValue"].shift(-1)

    # tag 뉴스 턴에서 해당 종목을 판매한 횟수
    bet_sell = bet_win.loc[(bet_win["newsTag"]==bet_win["riskLevel"]) & (bet_win["transactionType"]=="SELL")].copy()
    bet_sell_total = bet_sell.groupby("investSessionId")["nextValue"].count().reset_index().rename(columns={"nextValue":"bet_sell_total"})

    # tag 뉴스 턴에서 해당 종목을 판매하고 다음 턴에서 가격이 떨어진 횟수
    bet_sell["value_diff"] = bet_sell["nextValue"] - bet_sell["currentValue"]
    bet_sell_win = bet_sell[bet_sell["value_diff"]<0]
    bet_sell_win = bet_sell_win.groupby("investSessionId")["value_diff"].count().reset_index().rename(columns={"value_diff":"bet_sell_win"})

    # 성공 비율 계산
    bet_sell_df = pd.merge(bet_sell_total, bet_sell_win, on="investSessionId", how="left")
    bet_sell_df["betSellRatio"] = bet_sell_df["bet_sell_win"] / bet_sell_df["bet_sell_total"]


    # nan값 0으로 채우기
    bet_sell_df = bet_sell_df.fillna(0) # 데이터가 없어서 nan으로 출력됨..! 0으로 채워주기기

    # 필요없는 컬럼 삭제
    bet_sell_df.drop(columns=["bet_sell_total","bet_sell_win"], inplace=True)

    user_info = df.groupby("investSessionId")[["userId"]].first().reset_index()
    bet_sell_df = pd.merge(bet_sell_df, user_info, on="investSessionId", how="left")
    
    return bet_sell_df

betSellRatio = bet_sell_ratio(df)
betSellRatio.head()

,investSessionId,betSellRatio,userId
0,b'>N\xea\x92\x1eT\x04\xcb\xeeC\xf6V\xa6\xbd\x9...,1.0,956f51a8-d6a0-4a12-a22b-9da3cdffc879
1,"b""\x84L\xeb\xb7*>7\xdcu'+|\x95\xc6&\x93""",0.0,956f51a8-d6a0-4a12-a22b-9da3cdffc879
2,b'\x8fCj\x9a\xb0\x04[~2d\xb2\x96\xf9x\x12\xba',0.0,956f51a8-d6a0-4a12-a22b-9da3cdffc879
3,b'\x94@\x84\xdb\x006\x9c\xa5\xed\x84N\xadZs\xb...,0.0,f0220d43-513a-4619-973d-4ed84a42bf6a


# 데이터 병합

In [18]:
# 병합할 데이터프레임 리스트
dataframes_to_merge = [
    tradingTurn,
    transactionNum,
    avgCashRatio,
    avgStayTime,
    avgTradeRatio,
    tagAvgStayTime,
    betShares,
    betBuyRatio,
    betSellRatio
]

# 첫 번째 데이터프레임을 기준으로 시작
merged_final = dataframes_to_merge[0]

# 나머지 데이터프레임을 순회하며 outer merge 수행
for i in range(1, len(dataframes_to_merge)):
    merged_final = pd.merge(merged_final, dataframes_to_merge[i],
                            on=["investSessionId", "userId"],
                            how="outer")

merged_final.head()

,investSessionId,userId,avgNotKeep,click,avgCashRatio,avgStayTime,highBuyRatio,lowBuyRatio,midBuyRatio,highSellRatio,lowSellRatio,midSellRatio,tagTrunDuraion,betMidShares,betHighShares,betBuyRatio,betSellRatio
0,b'\x1cM\xa2\x9b\xc6\xa8\x90A\xd1\xbd\x13\xc8aE...,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,0.0,0.000000,-1.857143,17.500000,0.000000,0.000000,0.000000,0.00,0.0,0.00,0 days 00:00:24.750000,NaN,NaN,NaN,NaN
1,b' J\xe4QK<\x7f\xdb\x12K\xc2\x1c\xd3.\xa6\xa1',237aac1b-4d6f-4ca9-9e4f-30719ea5967d,1.0,1.066667,-0.753714,3.800000,0.500000,0.388889,0.111111,1.00,0.0,0.00,0 days 00:00:04,NaN,NaN,NaN,NaN
2,"b'""O\xcf1){(\x13\x9c_k[\xf4i\xcc\xa0'",f0220d43-513a-4619-973d-4ed84a42bf6a,1.0,0.944444,-0.235952,48.833333,0.588235,0.176471,0.235294,0.00,0.0,0.00,0 days 00:01:07.250000,4.0,4.0,0.333333,NaN
3,b'>N\xea\x92\x1eT\x04\xcb\xeeC\xf6V\xa6\xbd\x9...,956f51a8-d6a0-4a12-a22b-9da3cdffc879,1.0,1.222222,-0.548333,22.833333,0.166667,0.555556,0.277778,0.25,0.0,0.75,0 days 00:00:10.250000,4.0,2.0,0.666667,1.0
4,b'mB\xda\xce\xd4\x97\xfc\x83\xbd)\xa9 1t\x98\xab',237aac1b-4d6f-4ca9-9e4f-30719ea5967d,1.0,1.333333,0.680000,7.000000,0.000000,1.000000,0.000000,0.00,0.0,0.00,0 days 00:00:07,NaN,NaN,NaN,NaN


In [19]:
# child 와 병합
merged_final = pd.merge(merged_final, child, on="userId", how="left")
merged_final

,investSessionId,userId,avgNotKeep,click,avgCashRatio,avgStayTime,highBuyRatio,lowBuyRatio,midBuyRatio,highSellRatio,lowSellRatio,midSellRatio,tagTrunDuraion,betMidShares,betHighShares,betBuyRatio,betSellRatio,age,sex
0,b'\x1cM\xa2\x9b\xc6\xa8\x90A\xd1\xbd\x13\xc8aE...,237aac1b-4d6f-4ca9-9e4f-30719ea5967d,0.000000,0.000000,-1.857143,17.500000,0.000000,0.000000,0.000000,0.00,0.0,0.00,0 days 00:00:24.750000,NaN,NaN,NaN,NaN,24,M
1,b' J\xe4QK<\x7f\xdb\x12K\xc2\x1c\xd3.\xa6\xa1',237aac1b-4d6f-4ca9-9e4f-30719ea5967d,1.000000,1.066667,-0.753714,3.800000,0.500000,0.388889,0.111111,1.00,0.0,0.00,0 days 00:00:04,NaN,NaN,NaN,NaN,24,M
2,"b'""O\xcf1){(\x13\x9c_k[\xf4i\xcc\xa0'",f0220d43-513a-4619-973d-4ed84a42bf6a,1.000000,0.944444,-0.235952,48.833333,0.588235,0.176471,0.235294,0.00,0.0,0.00,0 days 00:01:07.250000,4.0,4.0,0.333333,NaN,24,M
3,b'>N\xea\x92\x1eT\x04\xcb\xeeC\xf6V\xa6\xbd\x9...,956f51a8-d6a0-4a12-a22b-9da3cdffc879,1.000000,1.222222,-0.548333,22.833333,0.166667,0.555556,0.277778,0.25,0.0,0.75,0 days 00:00:10.250000,4.0,2.0,0.666667,1.0,24,M
4,b'mB\xda\xce\xd4\x97\xfc\x83\xbd)\xa9 1t\x98\xab',237aac1b-4d6f-4ca9-9e4f-30719ea5967d,1.000000,1.333333,0.680000,7.000000,0.000000,1.000000,0.000000,0.00,0.0,0.00,0 days 00:00:07,NaN,NaN,NaN,NaN,24,M
5,b'oD?\xe6a$w\xc7Br\x02~c)\x89\xa3',f0220d43-513a-4619-973d-4ed84a42bf6a,1.000000,1.111111,-0.325238,6.000000,0.450000,0.000000,0.550000,0.00,0.0,0.00,0 days 00:00:06.500000,7.0,5.0,0.333333,NaN,24,M
6,b'\x83H}\xe9;\x86$<le\x02\xb9C\x116\xb0',f0220d43-513a-4619-973d-4ed84a42bf6a,1.000000,1.166667,-0.126190,13.166667,0.523810,0.047619,0.428571,0.00,0.0,0.00,0 days 00:00:08.500000,6.0,7.0,0.500000,NaN,24,M
7,"b""\x84L\xeb\xb7*>7\xdcu'+|\x95\xc6&\x93""",956f51a8-d6a0-4a12-a22b-9da3cdffc879,1.000000,1.444444,-0.101190,14.500000,0.190476,0.571429,0.238095,0.40,0.0,0.60,0 days 00:00:15,3.0,3.0,0.666667,0.0,24,M
8,b'\x8fCj\x9a\xb0\x04[~2d\xb2\x96\xf9x\x12\xba',956f51a8-d6a0-4a12-a22b-9da3cdffc879,0.833333,1.222222,0.127857,7.166667,0.157895,0.526316,0.315789,0.00,0.0,1.00,0 days 00:00:06.500000,4.0,2.0,0.666667,0.0,24,M
9,b'\x94@\x84\xdb\x006\x9c\xa5\xed\x84N\xadZs\xb...,f0220d43-513a-4619-973d-4ed84a42bf6a,1.000000,1.111111,-0.386429,18.166667,0.388889,0.222222,0.388889,1.00,0.0,0.00,0 days 00:00:17.750000,4.0,5.0,0.333333,0.0,24,M


In [20]:
merged_final.to_csv("./testdata1.csv", index=False)